# Dynamic casts - Any trait

In [30]:
use std::any::Any;

let boxed: Box<dyn Any> = Box::new(5);

let actual_id = (&*boxed).type_id();
let expected_id = std::any::TypeId::of::<i32>();

assert_eq!(actual_id, expected_id);

In [72]:
use std::fmt::Debug;

fn log<T: Any + Debug>(value: &T) {
    let value_any = value as &dyn Any;
    let type_name = std::any::type_name_of_val(value);
    
    match value_any.downcast_ref::<String>() {
        Some(as_string) => println!("Log: {}(len: {}): {}", type_name, as_string.len(), as_string),
        None => println!("Log: {}({:?})", type_name, value),
    }
}

log(&"Hello".to_string());
log(&42);
log(&vec![1, 2, 3]);

Log: alloc::string::String(len: 5): Hello
Log: i32(42)
Log: alloc::vec::Vec<i32>([1, 2, 3])


## Dynamic casts

In [73]:
trait Command: Any {
    fn execute(&self);

    fn as_any(&self) -> &dyn Any;
}

struct ACommand {
    message: String,
}

impl Command for ACommand {
    fn execute(&self) {
        println!("ACommand: {}", self.message);
    }

    fn as_any(&self) -> &dyn Any {
        self
    }
}

struct BCommand {
    text: String,
}

impl Command for BCommand {
    fn execute(&self) {
        println!("BCommand: {}", self.text);
    }

    fn as_any(&self) -> &dyn Any {
        self
    }
}

In [74]:
fn create_cmd(id: &'static str) -> Box<dyn Command> {
    match id {
        "A" => Box::new(ACommand {
            message: "Hello, World!".to_string(),
        }),
        "B" => Box::new(BCommand {
            text: "Hello, Rust!".to_string(),
        }),
        _ => panic!("Unknown command"),
    }    
}

In [79]:
let mut cmd = create_cmd("A");

assert!(cmd.as_any().is::<ACommand>());

match cmd.as_any().downcast_ref::<ACommand>() {
    Some(cmd) => cmd.execute(),
    None => println!("Unknown command"),
};

ACommand: Hello, World!
